In [11]:
from twisted.internet import defer

# Experiment 1
d = defer.Deferred()
print(d.called)
d.callback(3)
print(d.called)
print(d.result)

False
True
3


In [22]:
from twisted.internet import defer

d = defer.Deferred()

def foo(v):
    print("foo called")
    return v + 2

print(d.addCallback(foo))
print(d.called)
d.callback(4)
print(d.called)
print(d.result)


False
foo called
True
6


In [65]:
from twisted.internet import defer

# Experiment 3
def status(*ds):
    # print("status:")
    return [(getattr(d, 'result', "N/A"), len(d.callbacks)) for d in ds]

def b_callback(arg):
    print("b_callback called with arg =", arg)
    return b

def on_done(arg):
    print("on_done called with arg =", arg)
    return arg

# Experiment 3.a
a = defer.Deferred()
b = defer.Deferred()
a.addCallback(b_callback).addCallback(on_done)
print('status(a, b))两个延迟项都没有被触发，第一个有两个调回，第二个没有调回: \n', status(a, b), '\n')
print('a.callback:', a.callback(3), '\n')
print('status(a, b))先触发a, b_callback()已经被调回，只留下on_done() \n', status(a, b), '\n')
print('b.callback:', b.callback(4), '\n')
print('status(a, b)) last:', status(a, b), '\n')

# Experiment 3.b
a = defer.Deferred()
b = defer.Deferred()
a.addCallback(b_callback).addCallback(on_done)
print('b.callback: 先触发b:', b.callback(4), '\n')
print('status(a, b)):', status(a, b), '\n')
print('a.callback, 触发a b_callback on_done 都被调用:', a.callback(3), '\n')
print('status(a, b)) last:', status(a, b), '\n')

# Experiment 4
deferreds = [defer.Deferred() for i in range(5)]
join = defer.DeferredList(deferreds)
join.addCallback(on_done)
for i in range(4):
    deferreds[i].callback(i)

deferreds[4].callback(4)


status(a, b))两个延迟项都没有被触发，第一个有两个调回，第二个没有调回: 
 [('N/A', 2), ('N/A', 0)] 

b_callback called with arg = 3
a.callback: None 

status(a, b))先触发a, b_callback()已经被调回，只留下on_done() 
 [(<Deferred at 0x10d837278>, 1), ('N/A', 1)] 

on_done called with arg = 4
b.callback: None 

status(a, b)) last: [(4, 0), (None, 0)] 

b.callback: 先触发b: None 

status(a, b)): [('N/A', 2), (4, 0)] 

b_callback called with arg = 3
on_done called with arg = 4
a.callback, 触发a b_callback on_done 都被调用: None 

status(a, b)) last: [(4, 0), (None, 0)] 

on_done called with arg = [(True, 0), (True, 1), (True, 2), (True, 3), (True, 4)]


In [76]:
from time import sleep

# Hello, I'm a developer and I mainly setup Wordpress.
def install_wordpress(customer):
    # Our hosting company Threads Ltd. sucks. I start installation and...
    print("Start installation for", customer)

    # ...then wait till the installation finishes successfully. It is boring
    # and I'm spending most of my time waiting while consuming resources
    # (memory and some CPU cycles). It's because the process is *blocking*.
    sleep(2)
    print("All done for", customer)

# 单线程
def example1():
    # I do this all day long for our customers
    def developer_day(customers):
        for customer in customers:
            install_wordpress(customer)

    developer_day(["Bill", "Elon", "Steve", "Mark"])

print('# 单线程:\n')
example1()


# 多线程
def example2():
    import threading

    # The company grew. We now have many customers and I can't handle the
    # workload. We are now 5 developers doing exactly the same thing.
    def developers_day(customers):
        # But we now have to synchronize... a.k.a. bureaucracy
        lock = threading.Lock()

        def dev_day(id):
            print("Goodmorning from developer", id)
            # Yuck - I hate locks...
            lock.acquire()
            while customers:
                customer = customers.pop(0)
                lock.release()
                # My Python is less readable
                install_wordpress(customer)
                lock.acquire()
            lock.release()
            print("Bye from developer", id)

        # We go to work in the morning
        devs = [threading.Thread(target=dev_day, args=(i,)) for i in range(5)]
        [dev.start() for dev in devs]
        # We leave for the evening
        [dev.join() for dev in devs]

    # We now get more done in the same time but our dev process got more
    # complex. As we grew we spend more time managing queues than doing dev
    # work. We even had occasional deadlocks when processes got extremely
    # complex. The fact is that we are still mostly pressing buttons and
    # waiting but now we also spend some time in meetings.
    developers_day(["Customer %d" % i for i in range(15)])

print('\n# 5个线程: \n')
example2()

# 单线程:

Start installation for Bill


All done for Bill
Start installation for Elon


All done for Elon
Start installation for Steve


All done for Steve
Start installation for Mark


All done for Mark
# 多线程:

Goodmorning from developerGoodmorning from developer  1
Start installation for Customer 0
0Goodmorning from developer
Start installation for Customer 1
 2
Start installation for Customer 2
Goodmorning from developer 3
Start installation for Customer 3
Goodmorning from developer 4
Start installation for Customer 4


All done forAll done forAll done for Customer 0
Start installation for  Customer 1
Start installation for Customer 6 Customer 5

All done for Customer 3
Start installation for Customer 7
Customer 2
Start installation for Customer 8
All done for Customer 4
Start installation for Customer 9


All done forAll done for Customer 6
All done for All done for Customer 8
Start installation for Customer 10 Customer 5
Start installation for
 Customer 11
Start installation forCustomer 7
Start installation for Customer 13 Customer 12

All done for Customer 9
Start installation for Customer 14


All done forAll done for Customer 11 Customer 12
Bye from developer 2

Bye from developer 1
All done for Customer 10
Bye from developer 0All done for
 Customer 13
Bye from developer 3
All done for Customer 14
Bye from developer 4
